In [1]:
!pip install tldextract
!pip install xgboost
!pip install pandas numpy beautifulsoup4 textblob scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install nltk
!pip install wordcloud
!pip install scipy
!pip install nltk
!pip install wordcloud


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\email\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import re
import urllib.parse
from collections import Counter
import tldextract
import matplotlib.pyplot as plt
import numpy as np


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import time
import gc
import warnings
warnings.filterwarnings('ignore')

def load_data():
    print("Loading phishing email dataset...")
    df = pd.read_csv('final_phishing_dataset.csv')
    
    # Separate features and target variable
    X = df.drop('label', axis=1)
    y = df['label']
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of samples: {X.shape[0]}")
    print(f"\nClass distribution:")
    print(y.value_counts())
    
    return X, y

def evaluate_models(X, y):
    print("\n" + "="*60)
    print("MODEL EVALUATION AND COMPARISON")
    print("="*60)
    
    # Define machine learning models to test (SVM REMOVED - too slow for high dimensions)
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=300, solver='liblinear'),
        'Gaussian Naive Bayes': GaussianNB(),
        'Random Forest': RandomForestClassifier(n_estimators=15, random_state=42, max_depth=10, n_jobs=1),
        'XGBoost': XGBClassifier(n_estimators=15, random_state=42, max_depth=6, eval_metric='logloss', n_jobs=1)
    }
    
    # Set up cross-validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scaler = StandardScaler()
    
    # Store results
    results = {}
    best_model = None
    best_f1_score = 0
    
    print(f"Evaluating {len(models)} models using 3-fold cross-validation...")
    print(f"Total features used: {X.shape[1]}")
    print("-" * 60)
    
    for i, (model_name, model) in enumerate(models.items(), 1):
        print(f"\n[{i}/{len(models)}] Evaluating {model_name}...")
        start_time = time.time()
        
        # Create preprocessing and model pipeline
        pipeline = Pipeline([
            ('scaler', scaler),
            ('classifier', model)
        ])
        
        # Calculate different evaluation metrics
        metrics = ['f1_weighted', 'precision_weighted', 'recall_weighted', 'accuracy']
        model_results = {}
        
        for metric in metrics:
            try:
                print(f"  Computing {metric}...", end=" ")
                metric_start = time.time()
                scores = cross_val_score(pipeline, X, y, cv=cv, scoring=metric, n_jobs=1)
                metric_time = time.time() - metric_start
                
                mean_score = scores.mean()
                std_score = scores.std()
                
                model_results[metric] = {
                    'mean': mean_score,
                    'std': std_score,
                    'scores': scores
                }
                
                print(f"{mean_score:.4f} ({metric_time:.1f}s)")
                
            except Exception as e:
                print(f"ERROR: {str(e)[:50]}")
                model_results[metric] = {
                    'mean': np.nan,
                    'std': np.nan,
                    'scores': [np.nan] * 3
                }
        
        elapsed_time = time.time() - start_time
        print(f"{model_name} completed in {elapsed_time:.1f} seconds")
        
        results[model_name] = {
            'metrics': model_results,
            'model': model
        }
        
        # Track best performing model based on F1 score
        current_f1 = model_results.get('f1_weighted', {}).get('mean', 0)
        if current_f1 > best_f1_score and not np.isnan(current_f1):
            best_f1_score = current_f1
            best_model = model_name
        
        # Force garbage collection and brief pause
        gc.collect()
        time.sleep(0.5)
    
    return best_model, results

def print_results(best_model_name, all_results):
    print(f"\n{'='*60}")
    print("FINAL RESULTS AND MODEL COMPARISON")
    print(f"{'='*60}")
    
    # Sort models by F1 score performance
    sorted_results = sorted(
        all_results.items(), 
        key=lambda x: x[1]['metrics'].get('f1_weighted', {}).get('mean', 0), 
        reverse=True
    )
    
    print("\nModel Performance Ranking (sorted by F1 Score):")
    print("-" * 60)
    
    for rank, (model_name, results) in enumerate(sorted_results, 1):
        metrics = results['metrics']
        f1 = metrics.get('f1_weighted', {}).get('mean', np.nan)
        precision = metrics.get('precision_weighted', {}).get('mean', np.nan)
        recall = metrics.get('recall_weighted', {}).get('mean', np.nan)
        accuracy = metrics.get('accuracy', {}).get('mean', np.nan)
        
        print(f"{rank}. {model_name}:")
        print(f"F1-Score: {f1:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print()
    
    print(f"{'='*60}")
    print(f"BEST PERFORMING MODEL: {best_model_name}")
    
    best_metrics = all_results[best_model_name]['metrics']
    print(f"Best F1-Score: {best_metrics.get('f1_weighted', {}).get('mean', 'N/A'):.4f}")
    print(f"Best Precision: {best_metrics.get('precision_weighted', {}).get('mean', 'N/A'):.4f}")
    print(f"Best Recall: {best_metrics.get('recall_weighted', {}).get('mean', 'N/A'):.4f}")
    print(f"Best Accuracy: {best_metrics.get('accuracy', {}).get('mean', 'N/A'):.4f}")
    print(f"{'='*60}")


# Main execution
print("Phishing Email Detection - Machine Learning Model Comparison")
print("=" * 60)
X, y = load_data()
best_model_name, model_results = evaluate_models(X, y)
print_results(best_model_name, model_results)

print(f"\nAnalysis completed successfully!")
print(f"Total models evaluated: {len(model_results)}")
print(f"Cross-validation folds: 3")
print(f"Features used: {X.shape[1]} (all available features)")


Phishing Email Detection - Machine Learning Model Comparison
Loading phishing email dataset...
Dataset shape: (200609, 789)
Number of features: 789
Number of samples: 200609

Class distribution:
label
0.0    107534
1.0     93075
Name: count, dtype: int64

MODEL EVALUATION AND COMPARISON
Evaluating 4 models using 3-fold cross-validation...
Total features used: 789
------------------------------------------------------------

[1/4] Evaluating Logistic Regression...
  Computing f1_weighted... 0.9670 (130.1s)
  Computing precision_weighted... 0.9670 (136.4s)
  Computing recall_weighted... 0.9670 (129.7s)
  Computing accuracy... 0.9670 (136.2s)
Logistic Regression completed in 532.5 seconds

[2/4] Evaluating Gaussian Naive Bayes...
  Computing f1_weighted... 0.8370 (5.3s)
  Computing precision_weighted... 0.8397 (5.6s)
  Computing recall_weighted... 0.8377 (5.1s)
  Computing accuracy... 0.8377 (4.8s)
Gaussian Naive Bayes completed in 20.8 seconds

[3/4] Evaluating Random Forest...
  Computi

In [ ]:
# HYPERPARAMETER TUNING FOR BEST MODEL
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import time
import warnings
warnings.filterwarnings('ignore')

def load_dataset():
    """Load the phishing dataset for hyperparameter tuning"""
    print("Loading dataset for hyperparameter optimization...")
    df = pd.read_csv('final_phishing_dataset.csv')
    
    X = df.drop('label', axis=1)
    y = df['label']
    
    print(f"Dataset loaded: {X.shape[0]} samples, {X.shape[1]} features")
    return X, y

def scale_data(X_train, X_test):
    """Scale the training and test data"""
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

def evaluate_single_configuration(C, penalty, max_iter, X, y):
    """
    Evaluate a single hyperparameter configuration using 5-fold cross-validation
    """
    # Setup 5-fold cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store scores for each fold
    f1_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    
    # Perform 5-fold cross-validation manually
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        # Split data
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Scale the data
        X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)
        
        # Train the model
        model = LogisticRegression(
            C=C, 
            penalty=penalty, 
            max_iter=max_iter, 
            solver='liblinear', 
            random_state=42
        )
        model.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_scaled)
        
        # Calculate metrics
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
        recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    
    # Calculate mean and standard deviation
    results = {
        'f1_mean': np.mean(f1_scores),
        'f1_std': np.std(f1_scores),
        'accuracy_mean': np.mean(accuracy_scores),
        'accuracy_std': np.std(accuracy_scores),
        'precision_mean': np.mean(precision_scores),
        'precision_std': np.std(precision_scores),
        'recall_mean': np.mean(recall_scores),
        'recall_std': np.std(recall_scores)
    }
    
    return results

def hyperparameter_tuning(X, y):
    """
    Perform hyperparameter tuning for Logistic Regression using nested for loops
    """
    print("\n" + "="*70)
    print("HYPERPARAMETER TUNING - LOGISTIC REGRESSION")
    print("="*70)
    
    # Define hyperparameter values to test
    C_values = [0.01, 0.1, 1.0, 10.0, 100.0]
    penalty_values = ['l1', 'l2']
    max_iter_values = [300, 500, 1000]
    
    print("Hyperparameter search space:")
    print(f"C values: {C_values}")
    print(f"Penalties: {penalty_values}")
    print(f"Max iterations: {max_iter_values}")
    print(f"Total combinations: {len(C_values) * len(penalty_values) * len(max_iter_values)}")
    print(f"Using 5-fold cross-validation")
    print("-" * 70)
    
    # Store all results
    all_results = []
    best_f1_score = 0
    best_params = {}
    
    combination_count = 0
    total_combinations = len(C_values) * len(penalty_values) * len(max_iter_values)
    
    print("Starting hyperparameter optimization...")
    start_time = time.time()
    
    # Nested for loops to test all combinations
    for C in C_values:
        for penalty in penalty_values:
            for max_iter in max_iter_values:
                combination_count += 1
                
                print(f"Testing combination {combination_count}/{total_combinations}: C={C}, penalty={penalty}, max_iter={max_iter}")
                
                # Evaluate this configuration
                config_start_time = time.time()
                results = evaluate_single_configuration(C, penalty, max_iter, X, y)
                config_time = time.time() - config_start_time
                
                # Store results
                result_entry = {
                    'C': C,
                    'penalty': penalty,
                    'max_iter': max_iter,
                    'f1_score': results['f1_mean'],
                    'f1_std': results['f1_std'],
                    'accuracy': results['accuracy_mean'],
                    'precision': results['precision_mean'],
                    'recall': results['recall_mean'],
                    'time': config_time
                }
                all_results.append(result_entry)
                
                print(f"F1 Score: {results['f1_mean']:.4f} (+/- {results['f1_std']:.4f}) - Time: {config_time:.1f}s")
                
                # Check if this is the best configuration so far
                if results['f1_mean'] > best_f1_score:
                    best_f1_score = results['f1_mean']
                    best_params = {
                        'C': C,
                        'penalty': penalty,
                        'max_iter': max_iter
                    }
                    print(f"  ** New best configuration found! **")
                
                print()
    
    elapsed_time = time.time() - start_time
    print(f"Hyperparameter tuning completed in {elapsed_time:.1f} seconds")
    
    return all_results, best_params, best_f1_score

def analyze_tuning_results(all_results, best_params, best_f1_score):
    """
    Analyze and display the results of hyperparameter tuning
    """
    print("\n" + "="*70)
    print("HYPERPARAMETER TUNING RESULTS")
    print("="*70)
    
    # Display best parameters
    print("Best hyperparameters found:")
    print(f"  C: {best_params['C']}")
    print(f"  Penalty: {best_params['penalty']}")
    print(f"  Max iterations: {best_params['max_iter']}")
    print(f"\nBest cross-validation F1 score: {best_f1_score:.4f}")
    
    # Sort results by F1 score
    sorted_results = sorted(all_results, key=lambda x: x['f1_score'], reverse=True)
    
    # Display top 10 configurations
    print("\nTop 10 hyperparameter combinations:")
    print("-" * 70)
    
    for i in range(min(10, len(sorted_results))):
        result = sorted_results[i]
        print(f"{i+1}. F1 Score: {result['f1_score']:.4f} (+/- {result['f1_std']:.4f})")
        print(f"   C: {result['C']}, Penalty: {result['penalty']}, Max_iter: {result['max_iter']}")
        print(f"   Accuracy: {result['accuracy']:.4f}, Precision: {result['precision']:.4f}, Recall: {result['recall']:.4f}")
        print()
    
    return best_params

# Main execution for hyperparameter tuning
print("HYPERPARAMETER OPTIMIZATION FOR PHISHING DETECTION")
print("=" * 70)

# Load data
X, y = load_dataset()

# Perform hyperparameter tuning using nested for loops
all_results, best_params, best_f1_score = hyperparameter_tuning(X, y)

# Analyze results
best_params = analyze_tuning_results(all_results, best_params, best_f1_score)

print("\n" + "="*70)
print("HYPERPARAMETER TUNING COMPLETED")
print("="*70)
print("✅ Best hyperparameters found and validated")
print("✅ All evaluations performed using 5-fold cross-validation")  
print("✅ Model performance thoroughly tested")
print(f"🎯 Best parameters: C={best_params['C']}, penalty={best_params['penalty']}, max_iter={best_params['max_iter']}")
print(f"🎯 Expected F1-score: ~96.71%")
print("\n🔄 Next step: Run the next cell to train final model on complete dataset")


HYPERPARAMETER OPTIMIZATION FOR PHISHING DETECTION
Loading dataset for hyperparameter optimization...
Dataset loaded: 200609 samples, 789 features

HYPERPARAMETER TUNING - LOGISTIC REGRESSION
Hyperparameter search space:
C values: [0.01, 0.1, 1.0, 10.0, 100.0]
Penalties: ['l1', 'l2']
Max iterations: [300, 500, 1000]
Total combinations: 30
Using 5-fold cross-validation
----------------------------------------------------------------------
Starting hyperparameter optimization...
Testing combination 1/30: C=0.01, penalty=l1, max_iter=300
  F1 Score: 0.9638 (+/- 0.0009) - Time: 65.2s
  ** New best configuration found! **

Testing combination 2/30: C=0.01, penalty=l1, max_iter=500
  F1 Score: 0.9638 (+/- 0.0009) - Time: 58.1s

Testing combination 3/30: C=0.01, penalty=l1, max_iter=1000
  F1 Score: 0.9638 (+/- 0.0009) - Time: 57.2s

Testing combination 4/30: C=0.01, penalty=l2, max_iter=300
  F1 Score: 0.9670 (+/- 0.0006) - Time: 167.7s
  ** New best configuration found! **

Testing combinat

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pickle
import warnings
warnings.filterwarnings('ignore')

print("FINAL MODEL TRAINING FOR DEPLOYMENT")
print("=" * 50)

# Load dataset
df = pd.read_csv('final_phishing_dataset.csv')
X = df.drop('label', axis=1)
y = df['label']

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
print("Class distribution:")
print(y.value_counts())

# Best hyperparameters from tuning
best_params = {
    'C': 1.0,
    'penalty': 'l1', 
    'max_iter': 300
}

print(f"\nHyperparameters: C={best_params['C']}, penalty={best_params['penalty']}, max_iter={best_params['max_iter']}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train final model
final_model = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    max_iter=best_params['max_iter'],
    solver='liblinear',
    random_state=42
)

final_model.fit(X_scaled, y)

# Calculate training accuracy
train_predictions = final_model.predict(X_scaled)
train_accuracy = (train_predictions == y).mean()
print(f"Training accuracy: {train_accuracy:.4f}")

# Save model for Flask API
def save_model(model, scaler, params, filename='optimized_phishing_model.pkl'):
    model_package = {
        'model': model,
        'scaler': scaler,
        'parameters': params
    }
    
    try:
        with open(filename, 'wb') as f:
            pickle.dump(model_package, f)
        return True
    except Exception as e:
        print(f"Error saving model: {e}")
        return False

# Save and verify model
save_success = save_model(final_model, scaler, best_params)

if save_success:
    # Verify Flask API compatibility
    try:
        with open('optimized_phishing_model.pkl', 'rb') as f:
            test_load = pickle.load(f)
        
        required_keys = ['model', 'scaler', 'parameters']
        missing_keys = [key for key in required_keys if key not in test_load]
        
        if not missing_keys:
            print("Model saved successfully and verified for Flask API compatibility")
            print(f"Model type: {type(test_load['model']).__name__}")
            print(f"Scaler type: {type(test_load['scaler']).__name__}")
            print(f"Parameters: {test_load['parameters']}")
        else:
            print(f"Warning: Missing required keys: {missing_keys}")
            
    except Exception as e:
        print(f"Error verifying model: {e}")

    print("\nSUMMARY")
    print("-" * 30)
    print(f"Algorithm: Logistic Regression")
    print(f"Training samples: {X.shape[0]:,}")
    print(f"Features: {X.shape[1]}")
    print(f"Training accuracy: {train_accuracy:.4f}")
    print(f"Expected performance: ~96.71% F1-score")
    print(f"Model file: optimized_phishing_model.pkl")
    print("Flask API: Ready for deployment")
else:
    print("Model training completed but saving failed")


FINAL MODEL TRAINING FOR DEPLOYMENT
Dataset: 200609 samples, 789 features
Class distribution:
label
0.0    107534
1.0     93075
Name: count, dtype: int64

Hyperparameters: C=1.0, penalty=l1, max_iter=300
Training accuracy: 0.9689
Model saved successfully and verified for Flask API compatibility
Model type: LogisticRegression
Scaler type: StandardScaler
Parameters: {'C': 1.0, 'penalty': 'l1', 'max_iter': 300}

SUMMARY
------------------------------
Algorithm: Logistic Regression
Training samples: 200,609
Features: 789
Training accuracy: 0.9689
Expected performance: ~96.71% F1-score
Model file: optimized_phishing_model.pkl
Flask API: Ready for deployment
